In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.onnx 

from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

from torchvision.models.densenet import DenseNet201_Weights

In [ ]:

from typing import Any, Optional, Tuple

from torch import Tensor
import torch.nn.functional as F
class My_DenseNet(models.densenet.DenseNet):
    def forward(self, x: Tensor) -> Tensor:
        features = self.features(x)
        out = F.relu(features, inplace=True)
        # out = F.adaptive_avg_pool2d(out, (1, 1))
        # out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

def _densenet_my(
    growth_rate: int,
    block_config: Tuple[int, int, int, int],
    num_init_features: int,
    weights: Optional[models.densenet.WeightsEnum],
    progress: bool,
    **kwargs: Any,
) -> My_DenseNet:
    if weights is not None:
        models.densenet._ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = My_DenseNet(growth_rate, block_config, num_init_features, **kwargs)

    if weights is not None:
        models.densenet._load_state_dict(model=model, weights=weights, progress=progress)

    return model

@models.densenet.handle_legacy_interface(weights=("pretrained", DenseNet201_Weights.IMAGENET1K_V1))
def densenet201_my(*, weights: Optional[DenseNet201_Weights] = None, progress: bool = True, **kwargs: Any) -> My_DenseNet:
    r"""Densenet-201 model from
    `Densely Connected Convolutional Networks <https://arxiv.org/abs/1608.06993>`_.

    Args:
        weights (:class:`~torchvision.models.DenseNet201_Weights`, optional): The
            pretrained weights to use. See
            :class:`~torchvision.models.DenseNet201_Weights` below for
            more details, and possible values. By default, no pre-trained
            weights are used.
        progress (bool, optional): If True, displays a progress bar of the download to stderr. Default is True.
        **kwargs: parameters passed to the ``torchvision.models.densenet.DenseNet``
            base class. Please refer to the `source code
            <https://github.com/pytorch/vision/blob/main/torchvision/models/densenet.py>`_
            for more details about this class.

    .. autoclass:: torchvision.models.DenseNet201_Weights
        :members:
    """
    weights = DenseNet201_Weights.verify(weights)

    return _densenet_my(32, (6, 12, 48, 32), 64, weights, progress, **kwargs)

In [ ]:
model = densenet201_my(num_classes=1000, weights=DenseNet201_Weights.DEFAULT)

In [ ]:
# 原本創建 torch 內建 Model
# model = models.densenet201(num_classes=1000, pretrained=True) # 0.15 pretrained 參數會移除
# model = models.densenet201(num_classes=1000, weights=DenseNet201_Weights.DEFAULT) # 未來 # 雖然我還不知道 DenseNet201_Weights 變數怎麼 import

# 首先印出架構找出要替換的 Layer
# (本次的目的是要替換 FCN 成 segmentation layer)
# 因為 model 架構太大所以一定要用 ipynb 顯示
# print("net construct :", model)

In [ ]:
# 設定
input_size = (1, 480, 360)


In [ ]:
# 我先直接 segmentation 看能不能輸出
# model.classifier = nn.ConvTranspose2d(1920, 1, 4, stride=34, padding=0)
# model.classifier = nn.ConvTranspose2d(1920, 1, 4, stride=2, padding=1)
# model.classifier = nn.Sequential(
#     nn.ConvTranspose2d(1920, 1, 4, stride=34, padding=0), # [1, 1, 480, 344]
# )
model.classifier = nn.Sequential(
    nn.ConvTranspose2d(1920, 1, kernel_size=2, stride=2, padding=2),
    nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0),
    # nn.ConvTranspose2d(1, 1, kernel_size=3, stride=2, padding=1, output_padding=2),
    # nn.ConvTranspose2d(1, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
    # nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=2),
)
dummy_input = torch.randn(1, 3, 480, 360, requires_grad=True)

# from torch import Tensor
# def forward(self, x: Tensor) -> Tensor:
#     features = self.features(x)
#     out = F.relu(features, inplace=True)
#     out = F.adaptive_avg_pool2d(out, (1, 1))
#     out = torch.flatten(out, 1)
#     out = self.classifier(out)
#     return out

# model.forward = 

y = model(Variable(dummy_input))
print(y.shape)
# print("net construct :", model)

In [ ]:
# 測試是否能正常輸出

# set the model to inference mode 
# 不启用 Batch Normalization 和 Dropout (所以在做辨識之前一定要先呼叫這個)
# https://cloud.tencent.com/developer/article/1819853
model.eval() 

# Let's create a dummy input tensor  
# 這個是為了要讓 model 知道 input 大小是多少
dummy_input = torch.randn(1, 3, 480, 360, requires_grad=True)

# Export the model   
torch.onnx.export(model,         # model being run 
        dummy_input,       # model input (or a tuple for multiple inputs) 
        "densenet201.onnx",       # where to save the model  
        export_params=True,  # store the trained parameter weights inside the model file 
        opset_version=10,    # the ONNX version to export the model to 
        do_constant_folding=True,  # whether to execute constant folding for optimization 
        input_names = ['modelInput'],   # the model's input names 
        output_names = ['modelOutput'], # the model's output names 
        dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                            'modelOutput' : {0 : 'batch_size'}}) 

In [ ]:
# 測試 predict 輸出大小
# res = model.forward(dummy_input)
res = model(dummy_input)


In [ ]:
print(type(res))
print(len(res))
# print(res.mH)
# print(res.T)

In [ ]:
# 提取结果
pred = np.array(res.data.cpu()[0])[0]
# 处理结果
pred[pred >= 0.5] = 255
pred[pred < 0.5] = 0
# 保存图片
# cv2.imwrite(save_res_path, pred)
plt.imshow(pred)
